In [34]:
import numpy as np
import pandas as pd
import json
import os
import sys
import random
import PIL
import torch
import tqdm
import cv2
import re
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
from sklearn.metrics import confusion_matrix, accuracy_score

In [35]:
list_night_videos = ["5d2e49bd-2ff1-4ece-88af-baeb0d5bd05a",
"5ee7d32b-f829-4d0e-a6c8-49483361daee",
"85b695a1-c716-4550-a975-a0e9dbc3ea8a",
"89c41844-237d-4fa5-987e-58d76dda3e19",
"173e3d27-9047-4efe-868d-84a9a49a460f",
"249dd64f-89fe-468a-814f-8e437866a6e2",
"329b7816-d4b5-46f9-a475-f8ff3efbaf68",
"409b4dce-c55e-450f-bd1b-d1e7696592a9",
"8900f4f0-b19e-4cd7-84b0-7d6c73b1a226",
"34608613-c314-499d-b5f4-f602ef53b7c4",
"b511be77-9547-4420-8f7f-8c7d55635de3",
"bdb7d280-0700-4f79-871d-c32c03e944bb",
"c63e4048-596d-485b-a1e5-7f10b762631e",
"c3155b9d-d252-463a-bc6a-7360f923e5bc",
"cf55d35c-ca21-4a9d-b3d7-313b26c873ea",
"d72b7012-8139-41fa-ba15-d786bbfff33f",
"df5a2b24-b4dc-44ba-927d-e4665f3dd3af",
"e3b3c0ff-d95c-4764-8d97-1e3833fb90c1"]

In [22]:
def gather_annotations(annotation_folder):
    annotations = [elm for elm in os.listdir(annotation_folder) if not os.path.isdir(annotation_folder+elm)]
    columns = pd.read_excel(annotation_folder+annotations[0]).columns
    df = pd.DataFrame(columns=columns)
    for dataframe_name in annotations:
        dataframe = pd.read_excel(annotation_folder+dataframe_name)
        dataframe['Unnamed: 0'] = dataframe_name.replace(".xlsx","") + '-' + dataframe['Unnamed: 0'].astype(str)
        df = pd.concat([df,dataframe])
    df = df.rename(columns={'Unnamed: 0':'block_id'})
    return df

In [23]:
ground_truth = gather_annotations("data/ground_truth_spots_count/annotations_y/")

In [24]:
def rename_videos(videos_folder):
    vids = sorted(os.listdir(videos_folder))
    for n, vid in enumerate(vids) :
        print(n,vid)
        os.rename(video_folder + vid,f"{video_folder}video_{n+1}.mp4")

In [25]:
def split_video_into_blocks(video_path, sheet, target_folder):
    for _, row in sheet.iterrows():
        block_name = row['Unnamed: 0']
        start, end = str(row['start']).split(":"), str(row['end']).split(":")
        start = int(start[0]) * 60 + int(start[1])
        end =  int(end[0]) * 60 + int(end[1])
        ffmpeg_extract_subclip(video_path, start, end, targetname= target_folder + f"{block_name}.mp4")
        
        
    

In [5]:
def build_blocks_videos(videos_folder, ground_truth_folder):
    sheets = os.listdir(ground_truth_folder)
    for sheet_name in tqdm.notebook.tqdm(sheets):
        video_name = f"{sheet_name[:-5]}.mp4"
        video_path = videos_folder + video_name 
        target_folder = videos_folder + f'{sheet_name[:-5]}/'
        if not os.path.exists(target_folder):
            os.mkdir(target_folder)
        sheet = pd.read_excel(ground_truth_folder+sheet_name)
        sheet.dropna(axis=0,subset=['start','end'],inplace=True)
        split_video_into_blocks(video_path,sheet,target_folder,)
        

In [642]:
#build_blocks_videos("data/videos_h/","data/ground_truth_spots_count/annotations_h/")

  0%|          | 0/67 [00:00<?, ?it/s]

Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join

In [26]:
CLASSES = ['spot-1','spot-2','invalidSpot-fireHydrant','invalidspot-coloredCurb','invalidSpot-entrance','fireHydrant']

In [40]:
class CatchErrors : 
    def __init__(self, iou_threshold=0.2, min_persistance=5, conf_thres=None):
        self.iou_threshold = iou_threshold
        self.min_persistance = min_persistance
        self.conf_thres = conf_thres
    def bbox_iou(self,box1, box2,  eps=1e-7):
    # Returns the IoU of box1 to box2. box1 is 4, box2 is nx4
    
        b1_x1, b1_x2 = box1['bbox_left'] , box1['bbox_left'] + box1['bbox_w'] 
        b1_y1, b1_y2 = box1['bbox_top']  , box1['bbox_h'] + box1['bbox_top']
    
        b2_x1, b2_x2 = box2['bbox_left'] , box2['bbox_left'] + box2['bbox_w'] 
        b2_y1, b2_y2 = box2['bbox_top']  , box2['bbox_h'] + box2['bbox_top']

        # Intersection area
        inter = max(0, (min(b1_x2, b2_x2) - max(b1_x1, b2_x1)) ) * \
            max (0, (min(b1_y2, b2_y2) - max(b1_y1, b2_y1)) )

        # Union Area
        w1, h1 = b1_x2 - b1_x1, b1_y2 - b1_y1 + eps
        w2, h2 = b2_x2 - b2_x1, b2_y2 - b2_y1 + eps
        union = w1 * h1 + w2 * h2 - inter + eps

        iou = inter / union

        return iou  # IoU
    
    def class_switch_anomalie(self,frames):
        anomalies = {}
        for n, frame in enumerate(frames[1:]):
            precedent_frame = frames[n]
            if frame['object_id'] != precedent_frame['object_id'] and 'invalidspot' in frame['label'].lower() :
                iou = self.bbox_iou(frame,precedent_frame)
                if iou>0.20:
                    anomalies[str(precedent_frame['object_id'])] = iou
        return anomalies
    def confidence_anomalie(self,frames):
        if not self.conf_thres:
            return {}
        else:
            anomalies = {}
            for n, frame in enumerate(frames):
                if frame['confidence'] < self.conf_thres:
                    anomalies[str(frame['object_id'])] = frame['confidence']
            return anomalies
    def persistance_anomalie(self,frames):
        anomalies = {str(frame['object_id']) : 0 for frame in frames}
        for frame in frames:
            anomalies[str(frame['object_id'])] += 1
        return {key : value  for key, value in anomalies.items() if value < self.min_persistance}
        
            
        

In [65]:
len(os.listdir('data/merged_data/train/images/'))

35279

In [66]:
class CountSpots : 
    
    def __init__(self, video_path, txt_path, classes,min_persistance, iou_threshold,conf_thres,**kwargs):
        self.classes = classes
        self.video_path = video_path
        self.txt_path = txt_path
        self.preds = self.process_predictions()
        self.anomalie_detector = CatchErrors(iou_threshold=iou_threshold,min_persistance=min_persistance, conf_thres=conf_thres)
        
    
    def process_predictions(self):
        result = []
        with open(self.txt_path,'r') as f:
            for line in f.readlines():
                result.append( self.process_line( line ) )
        return result
    
    def get_dims_videos(self):
        vcap = cv2.VideoCapture(self.video_path) # 0=camera
        if vcap.isOpened(): 
            width = vcap.get(cv2.CAP_PROP_FRAME_WIDTH )
            height = vcap.get(cv2.CAP_PROP_FRAME_HEIGHT )
        return width, height
    
    def bbox_iou(self, box1, box2,  eps=1e-7):
        # Returns the IoU of box1 to box2. box1 is 4, box2 is nx4
    
        b1_x1, b1_x2 = box1['bbox_left'] , box1['bbox_left'] + box1['bbox_w'] 
        b1_y1, b1_y2 = box1['bbox_top']  , box1['bbox_h'] + box1['bbox_top']
    
        b2_x1, b2_x2 = box2['bbox_left'] , box2['bbox_left'] + box2['bbox_w'] 
        b2_y1, b2_y2 = box2['bbox_top']  , box2['bbox_h'] + box2['bbox_top']

        # Intersection area
        inter = max(0, (min(b1_x2, b2_x2) - max(b1_x1, b2_x1)) ) * \
            max (0, (min(b1_y2, b2_y2) - max(b1_y1, b2_y1)) )

    # Union Area
        w1, h1 = b1_x2 - b1_x1, b1_y2 - b1_y1 + eps
        w2, h2 = b2_x2 - b2_x1, b2_y2 - b2_y1 + eps
        union = w1 * h1 + w2 * h2 - inter + eps

        iou = inter / union

        return iou  # IoU
    
    def process_line(self, prediction):
    
        digit_regex = re.compile('\d+')
        numbers = digit_regex.findall(prediction)
        label = ""
        for elm in self.classes:
            if elm in prediction:
                label = elm
                break
        if label == "":
            if 'spot' in prediction and not 'invalid' in prediction:
                label="spot-1"
            
        
        frame_id, object_id = numbers[0], numbers[1]
        bbox_left, bbox_top = numbers[2], numbers[3]
        width, height = numbers[4], numbers[5]
        confidence = float(f"0.{numbers[-1]}")
        dictionary =  {'frame_id':int(frame_id),'object_id':int(object_id), 'label' : label,
           'bbox_left': int(bbox_left), 'bbox_top':int(bbox_top),
           'bbox_w':int(width), 'bbox_h':int(height),'confidence':confidence}
        return dictionary
    def process_blocks(self,series):
        center_w, center_h = self.get_dims_videos()
        center_w, center_h = center_w/2, center_h/2
        preds_left, preds_right = [], []
        for pred in series:
            if (pred['bbox_left']+pred['bbox_w']/2) < center_w:
                preds_left.append(pred)
            else:
                preds_right.append(pred)
        return preds_left, preds_right
    def split_by_series_of_frames(self):
        res = []
        serie = []
        for pred in self.preds:
            if len(serie) == 0:
                serie.append(pred)
            else:
                if pred['frame_id']-serie[-1]['frame_id'] < 5:
                    serie.append(pred)
                else:
                    res.append(serie)
                    serie = []
        return res
    def __count__(self,serie):
        switch_anomalies, persistance_anomalies = self.anomalie_detector.class_switch_anomalie(serie), self.anomalie_detector.persistance_anomalie(serie)
        conf_anomalies = self.anomalie_detector.confidence_anomalie(serie)
        res = set([])
        anomalies = dict(switch_anomalies,**persistance_anomalies)
        anomalies = dict(anomalies,**conf_anomalies).keys()
        for frame in serie:
            if frame['label'] in ['spot-1','spot-2'] and str(frame['object_id']) not in anomalies:
                res.add(frame['object_id'])
        return len(res)
    def count_preds(self):
        series = self.split_by_series_of_frames()
        left_count, right_count = 0, 0
        for serie in series:
            left_block, right_block = self.process_blocks(serie)
            left_count += self.__count__(left_block)
            right_count += self.__count__(right_block)
        return left_count, right_count
            

In [103]:
prediction_folder = "data/predictions_conf_0.35/"
video_folder= "data/videos/"
prediction_files = os.listdir(prediction_folder)
def build_predictions_dataframe(prediction_folder, video_folder, prediction_files,classes=CLASSES,
                                iou_threshold=0.2,min_persistance=4,conf_thres=0.4):
    df = {'block_id':[],'left':[],'right':[]}
    files = [f for f in os.listdir(prediction_folder) if 'checkpoint' not in f]
    for file_name in files:
        blocks_path = prediction_folder + file_name
        blocks = os.listdir(blocks_path)
        for block in blocks:
            if os.path.isdir(block):
                continue
            video_path = video_folder + file_name + f"/{block[:-3]}mp4"
            txt_path = prediction_folder+ f"{file_name}/{block}"
            counter =  CountSpots(video_path=video_path, txt_path=txt_path,
                    classes= CLASSES, iou_threshold=0.2, min_persistance=4,conf_thres=conf_thres)
            left, right = counter.count_preds()
            df['block_id'].append(f"{file_name}-{block[:-4]}")
            df['left'].append(left)
            df['right'].append(right)
    return pd.DataFrame(df)


In [140]:
def compute_score_dataframe(dataframe,exceptions):
    present_predictions = df['block_id'].tolist() 
    score_dataframe = {'block_id':[],'ground_truth_left':[], 'ground_truth_right':[],
                   'pred_left':[],'pred_right':[],
                       'invalidSpot-fireHydrant-right':[],'invalidSpot-fireHydrant-left':[]}
    for _, row in ground_truth.iterrows():
        block_id = row['block_id']
        video_name = block_id[:block_id.index("-BLOCK")]
        if video_name in exceptions:
            continue
        left, right = row['open spot left'], row['open spot right']
        if type(left) is str:
            left = 0
        if type(right) is str:
            right = 0
        if block_id in present_predictions:
            foo = df[ df['block_id']== block_id]
            left_, right_ = foo['left'].tolist()[0], foo['right'].tolist()[0]
            score_dataframe['block_id'].append(block_id)
            score_dataframe['ground_truth_left'].append(left)
            score_dataframe['ground_truth_right'].append(right)
            score_dataframe['pred_left'].append(left_)
            score_dataframe['pred_right'].append(right_)
            score_dataframe['invalidSpot-fireHydrant-right'].append(row['invalid (fire hydrant )spot right'])
            score_dataframe['invalidSpot-fireHydrant-left'].append(row['invalid (fire hydrant) spot left'])
            
    score_dataframe = pd.DataFrame(score_dataframe)
    for col in ['ground_truth_left', 'ground_truth_right', 'pred_left','pred_right']:
        score_dataframe[col] = score_dataframe[col].map(lambda x : 2 if int(x)>2 else int(x))
    return score_dataframe
    

In [141]:
from sklearn.metrics import classification_report

In [142]:
prediction_folder = "data/predictions_conf_0.42/"
df = build_predictions_dataframe(prediction_folder, video_folder, prediction_folder,conf_thres=None)
score_dataframe = compute_score_dataframe(df,exceptions=list_night_videos)
y, preds = score_dataframe['ground_truth_left'].tolist(), score_dataframe['pred_left'].tolist()
print(classification_report(y,preds,labels=[0,1,2],target_names=['no-spot','spot-1','spot-2']))
#right, day

              precision    recall  f1-score   support

     no-spot       0.30      0.91      0.46        23
      spot-1       0.12      0.11      0.12        18
      spot-2       1.00      0.20      0.33        55

    accuracy                           0.35        96
   macro avg       0.48      0.41      0.30        96
weighted avg       0.67      0.35      0.32        96



In [143]:
df = build_predictions_dataframe(prediction_folder, video_folder, prediction_folder,conf_thres=None)
score_dataframe = compute_score_dataframe(df,exceptions=list_night_videos)
y, preds = score_dataframe['ground_truth_right'].tolist(), score_dataframe['pred_right'].tolist()
print(classification_report(y,preds,labels=[0,1,2],target_names=['no-spot','spot-1','spot-2']))

              precision    recall  f1-score   support

     no-spot       0.58      0.79      0.67        38
      spot-1       0.14      0.25      0.18        12
      spot-2       0.78      0.39      0.52        46

    accuracy                           0.53        96
   macro avg       0.50      0.48      0.46        96
weighted avg       0.62      0.53      0.54        96

